In [1]:
%load_ext autoreload
%autoreload 2

from pricing.operations.functions import *
from pricing.operations.utils import *
from pprint import pprint
import os


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pricing.operations.experiments import level_1_kind_1_courses_1

In [11]:
df = spark.sql("""WITH base AS(
SELECT
cities.state AS state,
canonical_courses.name AS canonico,
AVG(maximum_value) AS wtp,
count(*) AS qtd
FROM
bi.global_users_searchs
JOIN
querobolsa_production.cities ON global_users_searchs.city_id = cities.id
JOIN
querobolsa_production.canonical_courses ON global_users_searchs.canonical_course_id = canonical_courses.id

WHERE
maximum_value <= 3000
AND
level_id = 1
AND
kind_id = 1
AND
global_users_searchs.canonical_course_id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)

GROUP BY
1,2

ORDER BY
1,3 desc
),
base_ordem AS(
SELECT
canonical_courses.name AS canonico,
AVG(maximum_value) AS wtp_geral

FROM
bi.global_users_searchs
JOIN
querobolsa_production.cities ON global_users_searchs.city_id = cities.id
JOIN
querobolsa_production.canonical_courses ON global_users_searchs.canonical_course_id = canonical_courses.id

WHERE
maximum_value <= 3000
AND
level_id = 1
AND
kind_id = 1
AND
global_users_searchs.canonical_course_id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)
AND
cities.state = 'PR'
AND
global_users_searchs.created_at >= '2019-10-01'

GROUP BY
1

ORDER BY
1,2 desc
)

SELECT
base.state,
base.canonico,
wtp,
qtd,
base_ordem.wtp_geral,
max(wtp) OVER
  (partition by base.canonico) AS max_wtp_canonico,
wtp/max(wtp) OVER
  (partition by base.canonico) AS relative_wtp

FROM
base
JOIN
base_ordem ON base_ordem.canonico = base.canonico

WHERE
qtd >= 100

ORDER BY
1 desc,5 desc""").toPandas()

fig = px.scatter(df, x='canonico', y='wtp', color='state')
fig.show()

In [12]:
df = spark.sql("""WITH base AS(
SELECT
cities.state AS state,
canonical_courses.name AS canonico,
AVG(maximum_value) AS wtp,
count(*) AS qtd
FROM
bi.global_users_searchs
JOIN
querobolsa_production.cities ON global_users_searchs.city_id = cities.id
JOIN
querobolsa_production.canonical_courses ON global_users_searchs.canonical_course_id = canonical_courses.id

WHERE
maximum_value <= 3000
AND
level_id = 1
AND
kind_id = 1
AND
global_users_searchs.canonical_course_id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)

GROUP BY
1,2

ORDER BY
1,3 desc
),
base_ordem AS(
SELECT
canonical_courses.name AS canonico,
AVG(maximum_value) AS wtp_geral

FROM
bi.global_users_searchs
JOIN
querobolsa_production.cities ON global_users_searchs.city_id = cities.id
JOIN
querobolsa_production.canonical_courses ON global_users_searchs.canonical_course_id = canonical_courses.id

WHERE
maximum_value <= 3000
AND
level_id = 1
AND
kind_id = 1
AND
global_users_searchs.canonical_course_id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)
AND
cities.state = 'PR'
AND
global_users_searchs.created_at >= '2019-10-01'

GROUP BY
1

ORDER BY
1,2 desc
)

SELECT
base.state,
base.canonico,
wtp,
qtd,
base_ordem.wtp_geral,
max(wtp) OVER
  (partition by base.canonico) AS max_wtp_canonico,
wtp/max(wtp) OVER
  (partition by base.canonico) AS relative_wtp

FROM
base
JOIN
base_ordem ON base_ordem.canonico = base.canonico

WHERE
qtd >= 100

ORDER BY
1 desc,5 desc""").toPandas()

fig = px.scatter(df, x='canonico', y='relative_wtp', color='state')
fig.show()

In [6]:
df = spark.sql("""WITH base as(
select distinct
    offers.university_id AS university_id,
    universities.name AS ies,
    offers.id AS offer_id,
    offers.course_id AS course_id,
    offers.discount_percentage,
    offers.visible,
    offers.offered_price,
    university_offers.full_price,
    campuses.city,
    campuses.state,
    courses.name AS curso,
    clean.name AS canonico,
    courses.kind,
    courses.level,
    courses.shift,
    pre_enrollment_fees.value,
    pre_enrollment_fees.original_value, 
    fee_experiment_alternatives.name AS alternative,
    fee_experiment_alternatives.id AS id_alternative,
    fee_experiment_alternatives.ratio,
    pre_enrollment_fees.value/university_offers.full_price AS pef_full,
    courses.digital_admission
    

from querobolsa_production.offers
    join querobolsa_production.university_offers on university_offers.id=offers.university_offer_id
    join querobolsa_production.pre_enrollment_fees on pre_enrollment_fees.offer_id=offers.id
    join querobolsa_production.courses on courses.id=offers.course_id
    join querobolsa_production.universities on universities.id = offers.university_id
    join querobolsa_production.levels on courses.level = levels.name
    join querobolsa_production.kinds on courses.kind = kinds.name
    join querobolsa_production.experiment_pre_enrollment_fees on pre_enrollment_fees.id = experiment_pre_enrollment_fees.pre_enrollment_fee_id
    join querobolsa_production.fee_experiment_alternatives on fee_experiment_alternatives.id = experiment_pre_enrollment_fees.fee_experiment_alternative_id
    join querobolsa_production.campuses ON courses.campus_id = campuses.id
    join querobolsa_production.canonical_courses can ON courses.canonical_course_id = can.id
    join querobolsa_production.canonical_courses clean ON clean.id = can.clean_canonical_course_id

where
    1=1
    and pre_enrollment_fees.enabled
    and offers.enabled
    and offers.visible
    and fee_experiment_alternatives.ratio <> 0
    and levels.parent_id = 1
    and kinds.parent_id = 1
    and fee_experiment_alternatives.name IN ('baseline')
    and clean.id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)
  
      
order by
1,17
),

base_ordem AS(
SELECT
canonico,
AVG(value) AS pef_geral
FROM
base
GROUP BY
1
)

SELECT
state,
base.canonico,
base_ordem.pef_geral,
AVG(value) AS pef,
AVG(original_value) AS pef_o


FROM
base
JOIN
base_ordem ON base.canonico = base_ordem.canonico

GROUP BY
1,2,3

ORDER BY
3 desc""").toPandas()

fig = px.scatter(df, x='canonico', y='pef', color='state')
fig.show()

In [14]:
df = spark.sql("""WITH base as(
select distinct
    offers.university_id AS university_id,
    universities.name AS ies,
    offers.id AS offer_id,
    offers.course_id AS course_id,
    offers.discount_percentage,
    offers.visible,
    offers.offered_price,
    university_offers.full_price,
    campuses.city,
    campuses.state,
    courses.name AS curso,
    clean.name AS canonico,
    courses.kind,
    courses.level,
    courses.shift,
    pre_enrollment_fees.value,
    pre_enrollment_fees.original_value, 
    fee_experiment_alternatives.name AS alternative,
    fee_experiment_alternatives.id AS id_alternative,
    fee_experiment_alternatives.ratio,
    pre_enrollment_fees.value/university_offers.full_price AS pef_full,
    courses.digital_admission
    

from querobolsa_production.offers
    join querobolsa_production.university_offers on university_offers.id=offers.university_offer_id
    join querobolsa_production.pre_enrollment_fees on pre_enrollment_fees.offer_id=offers.id
    join querobolsa_production.courses on courses.id=offers.course_id
    join querobolsa_production.universities on universities.id = offers.university_id
    join querobolsa_production.levels on courses.level = levels.name
    join querobolsa_production.kinds on courses.kind = kinds.name
    join querobolsa_production.experiment_pre_enrollment_fees on pre_enrollment_fees.id = experiment_pre_enrollment_fees.pre_enrollment_fee_id
    join querobolsa_production.fee_experiment_alternatives on fee_experiment_alternatives.id = experiment_pre_enrollment_fees.fee_experiment_alternative_id
    join querobolsa_production.campuses ON courses.campus_id = campuses.id
    join querobolsa_production.canonical_courses can ON courses.canonical_course_id = can.id
    join querobolsa_production.canonical_courses clean ON clean.id = can.clean_canonical_course_id

where
    1=1
    and pre_enrollment_fees.enabled
    and offers.enabled
    and offers.visible
    and fee_experiment_alternatives.ratio <> 0
    and levels.parent_id = 1
    and kinds.parent_id = 1
    and fee_experiment_alternatives.name IN ('baseline')
    and clean.id IN (2,10,14,34,48,115,121,140,153,170,192,211,214,217,257,288,869,341,346,356)
  
      
order by
1,17
),

base_ordem AS(
SELECT
canonico,
AVG(value) AS pef_geral
FROM
base
GROUP BY
1
),

base2 AS(
SELECT
state,
base.canonico,
base_ordem.pef_geral,
AVG(value) AS pef,
AVG(original_value) AS pef_o


FROM
base
JOIN
base_ordem ON base.canonico = base_ordem.canonico

GROUP BY
1,2,3

ORDER BY
1 desc,3 desc
)

SELECT
state,
canonico,
pef_geral,
pef,
pef_o,
MAX(pef) OVER
  (partition by canonico) AS max_pef,
pef/MAX(pef) OVER
  (partition by canonico) AS relative_pef

FROM
base2

ORDER BY 1 desc,3 desc""").toPandas()

fig = px.scatter(df, x='canonico', y='relative_pef', color='state')
fig.show()